<a href="https://colab.research.google.com/github/SDCraigN/UCB_DAB_Final_Project/blob/main/Machine_Learning_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
%load_ext google.colab.data_table

In [ ]:
%%bigquery --project plucky-zodiac-283501 df
SELECT *
FROM cleandata.combineddata

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (126 kB/s)
Reading package lists... Done


In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("tokenizing").getOrCreate()

In [ ]:
from pyspark.ml.feature import RegexTokenizer, Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [ ]:
# Read in data from storage
from pyspark import SparkFiles

url ="https://storage.googleapis.com/ucb_finalproject_nn/yearly_data/1975-1979.csv"
url2 = "https://storage.googleapis.com/ucb_finalproject_nn/yearly_data/1980-1989.csv"
url3 = "https://storage.googleapis.com/ucb_finalproject_nn/yearly_data/1990-2001.csv"
url4 = "https://storage.googleapis.com/ucb_finalproject_nn/yearly_data/2002-2009.csv"
url5 = "https://storage.googleapis.com/ucb_finalproject_nn/yearly_data/2010-2015.csv"
url6 = "https://storage.googleapis.com/ucb_finalproject_nn/yearly_data/2016_avg.csv"


In [ ]:
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("1975-1979.csv"), sep=",", header=True)

In [ ]:
spark.sparkContext.addFile(url2)
df2 = spark.read.csv(SparkFiles.get("1980-1989.csv"), sep=",", header=True)

In [ ]:
spark.sparkContext.addFile(url3)
df3 = spark.read.csv(SparkFiles.get("1990-2001.csv"), sep=",", header=True)

In [ ]:
spark.sparkContext.addFile(url4)
df4 = spark.read.csv(SparkFiles.get("2002-2009.csv"), sep=",", header=True)

In [ ]:
spark.sparkContext.addFile(url5)
df5 = spark.read.csv(SparkFiles.get("2010-2015.csv"), sep=",", header=True)

In [ ]:
spark.sparkContext.addFile(url6)
df6 = spark.read.csv(SparkFiles.get("2016_avg.csv"), sep=",", header=True)

In [ ]:
dfUnion = df.union(df2)

In [ ]:
dfUnion2 = dfUnion.union(df3)

In [ ]:
dfUnion3 = dfUnion.union(df4)

In [ ]:
dfUnion4 = dfUnion.union(df5)

In [ ]:
dfUnion5 = dfUnion.union(df6)

In [ ]:
dfUnion5.show()

+----+-----+-------+--------------------+------+--------+-------+-------------------+------------------+------------------+------------------+------------------+
|year|state|station|                name|   lat|     lon|   elev|           avg_temp|           avg_dew|           avg_slp|           avg_stp|           avg_co2|
+----+-----+-------+--------------------+------+--------+-------+-------------------+------------------+------------------+------------------+------------------+
|1975|   NH| 726160|BERLIN MUNICIPAL ...|44.583| -71.179|+0353.0| 48.552121212121207|40.676515151515147|5781.6575757575747|2755.7356060606062|329.85166666666669|
|1975|   NY| 725060|   OTIS ANGB AIRPORT| 41.65| -70.061|+0047.5| 54.339075268817204|45.689440860215058|            9999.9|1012.6343655913978|331.20300000000003|
|1975|   OR| 726885|             MEACHAM|45.511|-118.425|+1134.8| 40.082917735608305|3928.3748064282372|9102.0103225806452|  9087.61322580645|           331.521|
|1975|   AK| 700265|BARROW W

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
df_modified = dfUnion5.drop('name', 'lon', 'avg_slp', 'avg_stp', 'state', 'station')

df_modified.show()

+----+------+-------+-------------------+------------------+------------------+
|year|   lat|   elev|           avg_temp|           avg_dew|           avg_co2|
+----+------+-------+-------------------+------------------+------------------+
|1975|44.583|+0353.0| 48.552121212121207|40.676515151515147|329.85166666666669|
|1975| 41.65|+0047.5| 54.339075268817204|45.689440860215058|331.20300000000003|
|1975|45.511|+1134.8| 40.082917735608305|3928.3748064282372|           331.521|
|1975|71.333|+0004.0|               15.0|             12.25|            328.34|
|1975|70.233|-0999.0| 4.5260755682066796|5488.1165309603257|           332.224|
|1975|65.617|+0005.0| 14.700000000000001|10.966666666666665|            331.65|
|1975|69.367|+0085.0|-12.867118298368297|286.80541958041943|            329.53|
|1975|67.511|+0585.0|-10.939230769230772|748.59846153846138|           330.125|
|1975|  63.9|+0214.0|  28.33787867996201| 3323.198856328856|331.28999999999996|
|1975|  62.1|+0566.3| 27.595451631701632

In [ ]:
df_modified = df_modified.selectExpr("cast(year as int) year",
    "cast(lat as double) lat",
    "cast(elev as double) elev", 
    "cast(avg_temp as double) avg_temp",
    "cast(avg_dew as double) avg_dew", 
    "cast(avg_co2 as double) avg_co2")

df_modified.printSchema()

root
 |-- year: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- elev: double (nullable = true)
 |-- avg_temp: double (nullable = true)
 |-- avg_dew: double (nullable = true)
 |-- avg_co2: double (nullable = true)



In [ ]:
features = df_modified.drop("avg_temp")

In [ ]:
assembler = VectorAssembler(inputCols=features.columns, outputCol="features", handleInvalid="skip")

In [ ]:
output = assembler.transform(df_modified).select('features','avg_temp')

In [ ]:
train,test = output.randomSplit([0.75, 0.25]) 

In [ ]:
train.show()

+--------------------+------------------+
|            features|          avg_temp|
+--------------------+------------------+
|[1975.0,18.255,10...| 79.11708141321044|
|[1975.0,18.336,6....| 81.09575010118418|
|[1975.0,18.498,66...| 79.40661411822988|
|[1975.0,19.76,188...|58.107898479152425|
|[1975.0,20.001,81...|        66.9203125|
|[1975.0,20.79,399...| 71.07767868014932|
|[1975.0,21.155,13...| 76.06389411853509|
|[1975.0,21.324,2....| 75.23113799283155|
|[1975.0,21.45,7.3...| 75.23592165898617|
|[1975.0,21.487,25...| 72.82809042675284|
|[1975.0,21.984,30...| 75.04279185867894|
|[1975.0,22.033,7....| 75.93634641896564|
|[1975.0,24.583,1....| 78.99674859190989|
|[1975.0,24.633,1....| 77.21010822510823|
|[1975.0,24.72,1.0...| 77.56029761904762|
|[1975.0,25.483,2....| 75.74278673835124|
|[1975.0,25.914,7....| 73.02472862263184|
|[1975.0,26.072,3....| 78.24742962286138|
|[1975.0,26.184,30...| 73.21689132104456|
|[1975.0,26.228,11...| 77.92969779314889|
+--------------------+------------

In [ ]:
test.show()

+--------------------+-----------------+
|            features|         avg_temp|
+--------------------+-----------------+
|[1975.0,18.433,2....|79.68193484383002|
|[1975.0,19.719,11...|72.35902585765488|
|[1975.0,20.9,15.5...|74.15759408602152|
|[1975.0,21.317,15...|75.18506464413724|
|[1975.0,24.557,0....|79.18689260112649|
|[1975.0,24.95,1.0...|79.06531364468864|
|[1975.0,25.788,8....|76.77397593445983|
|[1975.0,26.77,1.0...|72.56022727272727|
|[1975.0,27.072,5....|             68.9|
|[1975.0,27.401,9....|74.34494815668202|
|[1975.0,27.6,1.0,...|            73.15|
|[1975.0,27.83,1.0...|68.96272104332449|
|[1975.0,27.911,3....| 75.3717601126472|
|[1975.0,27.962,5....|73.57108358934971|
|[1975.0,28.434,27...|71.91521761392728|
|[1975.0,29.692,50...|69.67728814644138|
|[1975.0,29.73,1.0...|64.38115100250626|
|[1975.0,30.067,5....| 68.5497977470558|
|[1975.0,30.321,20...|66.64256656426012|
|[1975.0,30.4,4.9,...|71.40381995540692|
+--------------------+-----------------+
only showing top

In [ ]:
from pyspark.ml.regression import LinearRegression
lin_reg = LinearRegression(featuresCol = 'features', labelCol='avg_temp')
linear_model = lin_reg.fit(train)
linear_model2 = lin_reg.fit(test)

In [ ]:
trainSummary = linear_model.summary
print("RMSE: %f" % trainSummary.rootMeanSquaredError)
print("\nr2: %f" % trainSummary.r2)

RMSE: 6.073987

r2: 0.763184


Root MSE: absolute measure of fit. Standard deviation of unexplained variance / square root of the variance of the residuals. 

R-squared: Relative measure of fit. Approximately three-quarters of the observed variation can be explained by the model's inputs. 

In [ ]:
testSummary = linear_model2.summary
print("RMSE: %f" % testSummary.rootMeanSquaredError)
print("\nr2: %f" % testSummary.r2) # 74.5202% 

RMSE: 6.267120

r2: 0.745202


Root MSE: absolute measure of fit. standard deviation of unexplained variance / square root of the variance of the residuals. 

R-squared: relative measure of fit. approximately three-quarters of the observed variation can be explained by the model's inputs.

Validation: Want to understand how our model performs on new data:
  You fit (train) the model using training data, and score and validate your model using the testing data.

Residual plot: checks our predictions, want y-values to be close to 0, and want them to be random -- no pattern!
